In [1]:
import numpy as np
from simplifiedLikelihoods import Data,UpperLimitComputer
import pandas as pd

### Define MET bins and their order

In [2]:
metBins = [250,  280,  310,  340,  370,  400,  430,  470,  510, 550,  590,  640,  690,  
            740,  790,  840,  900,  960, 1020, 1090, 1160, 1250, 1400]

### Load CMS data

In [3]:
cmsYieldFile = '../AuxInfo/CMS-EXO-20-004-data/HEPData-ins1894408-v2-csv/Simplifiedlikelihood:Yields(Monojet).csv'
bgYields = np.genfromtxt(cmsYieldFile,delimiter=',',skip_header=5,
                          skip_footer=67,names=True,dtype=None,encoding=None)

dataYields = np.genfromtxt(cmsYieldFile,delimiter=',',skip_header=73,
                           names=True,dtype=None,encoding=None)

covarianceFile = '../AuxInfo/CMS-EXO-20-004-data/HEPData-ins1894408-v2-csv/Simplifiedlikelihood:covariancematrix(Monojet).csv'
cov = np.genfromtxt(covarianceFile,delimiter=',',skip_header=5,
                           names=True,dtype=None,encoding='UTF-8')

### Extract covariance matrix

In [4]:
srOrder = cov['Second_Bin'][:66]
covMatrix = np.zeros((len(srOrder),len(srOrder)))
for pt in cov:
    i = np.where(srOrder == pt['First_Bin'])
    j = np.where(srOrder == pt['Second_Bin'])
    covMatrix[i,j] = pt['Covariance']

### Get number of observed and expected (BG) events

In [5]:
nobs = [int(dataYields[dataYields['Bin'] == sr]['Data_yield'][0]) for sr in srOrder]
nbg = [float(bgYields[bgYields['Bin'] == sr]['Background_yield'][0]) for sr in srOrder]

### Load Recast Data

In [6]:
recastFile = '../scanResults.pcl'
recastData = pd.read_pickle(recastFile)

### Get all model points

In [7]:
models = []
mCols = ['Coupling','Mode','$m_{med}$','$m_{DM}$']
for row in recastData[mCols].values:
    m = dict(zip(mCols,row.tolist()))
    if m not in models:
        models.append(m)
print(len(models))

[{'Coupling': 'Axial', 'Mode': 'DM+QCDjets', '$m_{med}$': 100.0, '$m_{DM}$': 1.0}, {'Coupling': 'Axial', 'Mode': 'DM+QCDjets', '$m_{med}$': 100.0, '$m_{DM}$': 30.0}, {'Coupling': 'Axial', 'Mode': 'DM+QCDjets', '$m_{med}$': 100.0, '$m_{DM}$': 40.0}, {'Coupling': 'Axial', 'Mode': 'DM+QCDjets', '$m_{med}$': 100.0, '$m_{DM}$': 45.0}, {'Coupling': 'Axial', 'Mode': 'DM+QCDjets', '$m_{med}$': 100.0, '$m_{DM}$': 55.0}, {'Coupling': 'Axial', 'Mode': 'DM+QCDjets', '$m_{med}$': 100.0, '$m_{DM}$': 60.0}, {'Coupling': 'Axial', 'Mode': 'DM+QCDjets', '$m_{med}$': 100.0, '$m_{DM}$': 70.0}, {'Coupling': 'Axial', 'Mode': 'DM+QCDjets', '$m_{med}$': 100.0, '$m_{DM}$': 100.0}, {'Coupling': 'Axial', 'Mode': 'DM+QCDjets', '$m_{med}$': 100.0, '$m_{DM}$': 150.0}, {'Coupling': 'Axial', 'Mode': 'DM+QCDjets', '$m_{med}$': 100.0, '$m_{DM}$': 200.0}, {'Coupling': 'Axial', 'Mode': 'DM+QCDjets', '$m_{med}$': 250.0, '$m_{DM}$': 1.0}, {'Coupling': 'Axial', 'Mode': 'DM+QCDjets', '$m_{med}$': 250.0, '$m_{DM}$': 75.0}, {'

### Loop over model points and compute UL on mu

In [8]:
ulComp = UpperLimitComputer()

for m in models:
    dfModel = recastData.loc[(recastData[list(m)] == pd.Series(m)).all(axis=1)]
    ns = []
    for sr in srOrder:
        dataset,ibin = sr.split('_')[1:]
        dataset = eval(dataset)
        ibin = eval(ibin.replace('bin',''))
        binLabel = 'bin_%1.1f_%1.1f' %(metBins[ibin],metBins[ibin+1])
        signalYield = dfModel[dfModel['Data-takingperiod'] == dataset][binLabel].iloc[0]
        ns.append(signalYield)
    ns = np.array(ns)    
    data = Data(observed=nobs, backgrounds=nbg, covariance=covMatrix, 
                nsignal=ns,deltas_rel=0.0)
    dataExp = Data(observed=[int(b) for b in nbg], backgrounds=nbg, covariance=covMatrix, 
                   nsignal=ns,deltas_rel=0.0)    
    
    ul = ulComp.getUpperLimitOnMu(data)
    ulExp = ulComp.getUpperLimitOnMu(dataExp)    
    recastData.loc[dfModel.index,'$\mu^{UL}_{obs}$'] = ul
    recastData.loc[dfModel.index,'$\mu^{UL}_{exp}$'] = ulExp

In [9]:
recastData

,Coupling,Mode,$m_{med}$,$m_{DM}$,Data-takingperiod,Luminosity (1/fb),bin_250.0_280.0,bin_250.0_280.0_ErrorPlus,bin_250.0_280.0_ErrorMinus,bin_280.0_310.0,...,Tauveto,Bjetveto,Photonveto,"$\Delta \phi (jet,p_{T}^{miss})>0.5$ rad",LeadingAK4jet$p_{T}>100$GeV,LeadingAK4jet$\eta<2.4$,HCALmitigation(jets),HCALmitigation($\phi^{miss}$),$\mu^{UL}_{obs}$,$\mu^{UL}_{exp}$
0,Axial,DM+QCDjets,100.0,1.0,2018,59.7,245267.033817,37712.097224,37712.097224,167566.730775,...,0.114114,0.107835,0.107032,0.107032,0.105773,0.105773,0.102218,0.086625,0.007005,0.004253
1,Axial,DM+QCDjets,100.0,1.0,2017,41.5,171037.028539,31648.319020,31648.319020,144064.647522,...,0.108655,0.099589,0.098690,0.098690,0.098690,0.098690,0.098690,0.098690,0.007005,0.004253
2,Axial,DM+QCDjets,100.0,1.0,2016,36.0,200355.046482,35688.125801,35688.125801,158606.033063,...,0.116778,0.104304,0.103335,0.103335,0.103335,0.103335,0.103335,0.103335,0.007005,0.004253
3,Axial,DM+QCDjets,100.0,30.0,2018,59.7,237337.540589,30585.458654,30585.458654,87023.598064,...,0.117735,0.106724,0.104588,0.104588,0.104588,0.103294,0.101075,0.086948,0.012456,0.007563
4,Axial,DM+QCDjets,100.0,30.0,2017,41.5,152203.603754,24589.755037,24589.755037,101331.647581,...,0.125928,0.112970,0.111483,0.109671,0.104760,0.103865,0.103865,0.103865,0.012456,0.007563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,Axial,DM+QCDjets,2500.0,1000.0,2017,41.5,14.232039,1.241442,1.241442,10.003953,...,0.371021,0.322961,0.310816,0.310352,0.308471,0.306797,0.306797,0.306797,14.635416,7.311840
125,Axial,DM+QCDjets,2500.0,1000.0,2016,36.0,12.857400,1.150184,1.150184,9.678348,...,0.371482,0.324070,0.316453,0.316322,0.311872,0.311557,0.311557,0.311557,14.635416,7.311840
126,Axial,DM+QCDjets,2500.0,1500.0,2018,59.7,0.025180,0.003968,0.003968,0.025579,...,0.384380,0.320049,0.303547,0.304565,0.301983,0.300111,0.286656,0.248021,3745.870728,3410.931879
127,Axial,DM+QCDjets,2500.0,1500.0,2017,41.5,0.025427,0.003832,0.003832,0.011817,...,0.360850,0.317088,0.307916,0.307916,0.304112,0.302716,0.302716,0.302716,3745.870728,3410.931879


In [11]:
recastData.to_pickle(recastFile)